# Download Mobi Bike Share Data

- Mobi makes several years of trip data available at [on their website](https://www.mobibikes.ca/en/system-data).
- They also adhere to the [GBFS standard](INSERT APPROPRIATE LINK) and have an API available to retrieve live status about stations on the network.
- We will also scrape the [Mobi website](INSERT LINK) in case the data is useful to enrich our product later on.

In this notebook, we will:
1. Download and process available trip history (2018-2025, ~7.6M trips).
2. Create bronze tables for trips, stations and site data with minimal data maniupulation.
3. Examine the contents of the bronze tables and cleanse as appropriate to create silver tables.

Runs on Databricks Unity Catalog (Serverless or cluster).

## Setup and Configuration

Before running this notebook, ensure your `config.yaml` is correctly configured with:

- `catalog` — your Unity Catalog catalog  
- `schema` — the schema you will write tables into  
- `raw_data_vol` — the UC volume where raw data files will be stored  
- Optional scraper parameters for Mobi website content  

This notebook expects that:

- You are using **Databricks Serverless**
- You are working entirely inside **Unity Catalog**
- All data is stored using **UC Volumes** and **Delta Tables**

### Overwrite Behavior

Many cells in this notebook use **overwrite mode** when writing into the RAW volume or Delta tables.

This is intentional and safe for the hackathon:

- If you re-download the data or fix a transformation, **overwrite ensures your tables stay consistent**.
- It prevents half-written or mixed-version datasets caused by incremental writes.
- It allows teams to restart from scratch without manually clearing volumes or tables.

**If you want to preserve an older version**, simply rename the table before re-running.

During the hackathon, feel free to run this notebook multiple times — it is designed to be **idempotent** and safe to rerun.


In [0]:
%pip install requests pandas pyarrow beautifulsoup4 openpyxl mlflow markdownify loguru

%restart_python

In [0]:
# Notebook bootstrap: autoreload, src path, imports
%reload_ext autoreload
%autoreload 2

In [0]:
import sys
from pathlib import Path
from urllib.parse import urlparse

# Add the repository's src directory using a relative path
src_path = Path.cwd() / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import mlflow
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from loguru import logger
from pyspark.sql import Window
from pyspark.sql import functions as F


from src.mobi import (
    BasicSiteScraper,
    MobiDataDownloaderError,
    StationDataError,
    combine_trip_data,
    download_all_trip_data,
    fetch_station_info_from_gbfs,
    fetch_station_status_from_gbfs,
    seed_volume_from_backup,
    save_to_parquet,
)

CONFIG = mlflow.models.ModelConfig(development_config="config.yaml")

CATALOG = CONFIG.get("catalog")
SCHEMA = CONFIG.get("schema")
RAW_VOLUME = CONFIG.get("raw_data_vol")
SHUFFLE_PARTITIONS = CONFIG.get("shuffle_partitions")

logger.remove()
logger.add(
    sink=sys.stderr,
    level="INFO",
    format="{time:YYYY-MM-DD HH:mm:ss} | {level:<8} | {message}",
)
logger = logger.bind(notebook="01_data")

In [0]:
OVERWRITE_DOWNLOADS = bool(CONFIG.get("overwrite_downloads"))
BUNDLE_PATH = Path.cwd() / "data.zip"
logger.info(f"OVERWRITE_DOWNLOADS set to {OVERWRITE_DOWNLOADS}")



In [0]:
# Project config & UC paths

# Ensure catalog, schema, and volume exist; create if missing
# TODO: add check for catalog
try:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA}`")
except Exception as e:
    logger.warning(f"Have you created the catalog? {CATALOG}")
    logger.error(f"Error creating schema: {e}")

spark.sql(f"CREATE VOLUME IF NOT EXISTS `{CATALOG}`.`{SCHEMA}`.`{RAW_VOLUME}`")

# UC Volume root (used by pandas and Spark)
volume_root = Path("/Volumes") / CATALOG / SCHEMA / RAW_VOLUME
trip_data_dir = volume_root / "trip_data"
site_data_dir = volume_root / "mobi_site"

# Spark tuning (shared across all writes)
spark.conf.set("spark.sql.shuffle.partitions", SHUFFLE_PARTITIONS)

logger.info("Using Unity Catalog volume storage")
logger.info(f"catalog.schema = {CATALOG}.{SCHEMA}")
logger.info(f"volume = {RAW_VOLUME}")
logger.info(f"spark.sql.shuffle.partitions = {SHUFFLE_PARTITIONS}")

seed_volume_from_backup(volume_root, BUNDLE_PATH)
logger.info("Seeded volume with bundled trip and site data (non-destructive)")

## Download Trip Data

Here we pull historical trip data from the public Mobi open data portal and store it inside your configured **RAW** UC volume.

This step:

- Retrieves all published trip CSVs.
- Ensures consistent naming and storage inside UC.
- Falls back to a bundled archive if downloads are unavailable (e.g., network issues during the hackathon).

Once this completes, you should:

- Browse the RAW volume to confirm the CSV files exist.
- Validate a few rows using `spark.read.csv` if desired.


In [0]:
# Download all files
raw_trip_dir = trip_data_dir / "raw"
raw_trip_dir.mkdir(parents=True, exist_ok=True)

if OVERWRITE_DOWNLOADS:
    logger.info(f"Downloading trip data to {raw_trip_dir}")
    files = download_all_trip_data(raw_trip_dir, overwrite=True)
    logger.success(f"Downloaded {len(files)} files")
else:
    logger.info(
        "OVERWRITE_DOWNLOADS is False; using pre-seeded trip extracts instead of downloading"
    )
    files = sorted(raw_trip_dir.glob("*.csv"))
    if not files:
        raise RuntimeError(
            "No trip CSV files available in the seeded backup. Set OVERWRITE_DOWNLOADS=True to fetch fresh data."
        )
    logger.success(f"Found {len(files)} pre-seeded trip files")


In [0]:
# Process and combine
logger.info("Combining downloaded trip extracts")
trips_df = combine_trip_data(files)
logger.success(f"Total trips processed: {len(trips_df):,}")

# Save
trips_output = trip_data_dir / "mobi_trips.parquet"
logger.info(f"Writing trips parquet to {trips_output}")
save_to_parquet(trips_df, trips_output)
logger.success("Trips parquet saved")

## Download Station Data

Next we turn to **station-level data** using GBFS (General Bikeshare Feed Specification), the standard for bike share system metadata and status feeds. 🚲

In this section you will:

- Call the Mobi GBFS endpoints to retrieve:
  - **Station information** (IDs, names, locations, capacities).
  - **Station status** (bikes available, docks available, etc.).
- Combine inventory and status so you know both *where* stations are and *what’s going on* at each one.
- Persist these snapshots into your RAW volume or directly into a bronze Delta table.

This gives your team:

- A foundation for **availability modelling**, **rebalancing logic**, or **real-time insights**.
- Station attributes that you can join to trip data and scraped site content.

> Data quality is part of the challenge: check for missing values, unusual stations, or files that don’t match the expected format.


In [0]:
# Fetch from GBFS API
logger.info("Preparing station inventory and status data")
stations_parquet = trip_data_dir / "mobi_stations.parquet"
stations_csv = trip_data_dir / "mobi_stations.csv"

if OVERWRITE_DOWNLOADS:
    logger.info("OVERWRITE_DOWNLOADS is True; fetching stations from GBFS")
    station_info = fetch_station_info_from_gbfs()
    station_status = fetch_station_status_from_gbfs()
    stations = station_info.merge(station_status, on="station_id", how="left")
    logger.success(f"Merged station details: {len(stations)} rows")

    logger.info(f"Writing stations parquet to {stations_parquet}")
    save_to_parquet(stations, stations_parquet)
    logger.info(f"Exporting stations CSV to {stations_csv}")
    stations.to_csv(stations_csv, index=False)
    logger.success("Station datasets updated")
else:
    logger.info(
        "OVERWRITE_DOWNLOADS is False; reusing pre-seeded station snapshot"
    )
    if stations_parquet.exists():
        stations = pd.read_parquet(stations_parquet)
    elif stations_csv.exists():
        stations = pd.read_csv(stations_csv)
    else:
        raise RuntimeError(
            "No station snapshot found in the seeded backup. Set OVERWRITE_DOWNLOADS=True to fetch fresh data."
        )
    logger.success(f"Loaded station snapshot with {len(stations)} rows")


## Scrape Mobi Website

Many interesting questions aren’t just about numbers — they’re about how the system is described to riders and stakeholders.

In this section we:

- Optionally scrape selected pages from **https://www.mobibikes.ca/**.
- Extract page titles, URLs, and main content into a structured table.
- Prepare this content for use in **Vector Search** and **Genie rooms** (e.g. FAQ-style Q&A, policy lookups, marketing copy).

Typical uses for this content in the hackathon:

- Power a Genie room that can answer “how does Mobi handle X?” using first-party content.
- Enrich trip or station analyses with qualitative context from the website.

> Scraping is intentionally lightweight and focused. You’re welcome to extend it, but keep an eye on depth and performance.


In [0]:
# Configure scraper
BASE_URL = CONFIG.get("scrape_url")
START_URL = BASE_URL
SCRAPE_DELAY = float(CONFIG.get("scrape_delay"))
SCRAPE_MAX_DEPTH = int(CONFIG.get("scrape_max_depth"))

scraper = BasicSiteScraper(
    base_url=BASE_URL,
    delay=SCRAPE_DELAY,
    max_depth=SCRAPE_MAX_DEPTH,
)

logger.info(f"Scraping mobi site starting at {START_URL}")

if OVERWRITE_DOWNLOADS:
    pages = scraper.scrape_recursive(START_URL)
    logger.success(f"Pages scraped: {len(pages)}")
else:
    logger.info("OVERWRITE_DOWNLOADS is False; loading bundled site markdown")
    fallback_root = site_data_dir / "raw"
    fallback_files = sorted(fallback_root.glob("*.md"))
    if not fallback_files:
        raise RuntimeError(
            "No site markdown files available in the seeded backup. Set OVERWRITE_DOWNLOADS=True to scrape fresh content."
        )

    pages = {}
    for path in fallback_files:
        content = path.read_text(encoding="utf-8")
        url = f"file:///{path.name}"
        pages[url] = {
            "metadata": {
                "title": path.stem.replace("_", " ").title(),
                "description": "",
                "main_heading": "",
                "url": url,
                "scraped_at": None,
            },
            "content": content,
            "links": [],
            "status": "fallback",
        }

    logger.success(f"Loaded {len(pages)} pages from bundled markdown")

preview_urls = list(pages.keys())[: min(10, len(pages))]
logger.info(f"Previewing first {len(preview_urls)} URLs: {preview_urls}")


In [0]:
# Save markdown content per page
raw_site_data_dir = site_data_dir / "raw"
raw_site_data_dir.mkdir(parents=True, exist_ok=True)
logger.info(f"Persisting raw site markdown to {raw_site_data_dir}")

def url_to_filename(url: str) -> str:
    parsed = urlparse(url)
    path = parsed.path.rstrip("/") or "index"
    safe = re.sub(r"[^a-zA-Z0-9._-]+", "_", path)
    if not safe.endswith(".md"):
        safe += ".md"
    return safe

count = 0
for url, data in pages.items():
    filepath = raw_site_data_dir / url_to_filename(url)
    filepath.write_text(data.get("content", ""), encoding="utf-8")
    count += 1

logger.success(f"Wrote {count} markdown files to {raw_site_data_dir}")

# Save to parquet
records = []
for url, data in pages.items():
    md = data.get("metadata", {})
    records.append({
        "url": url,
        "title": md.get("title", ""),
        "description": md.get("description", ""),
        "main_heading": md.get("main_heading", ""),
        "scraped_at": md.get("scraped_at", None),
        "status": data.get("status", ""),
        "content_md": data.get("content", ""),
    })

df = pd.DataFrame.from_records(records)
output_path = site_data_dir / "mobibikes_ca_content.parquet"
logger.info(f"Saving site content parquet to {output_path}")
save_to_parquet(df, output_path)
logger.success("Site content parquet saved")

## Load and Explore

Once the trip CSVs are in your RAW volume, we load them into Spark and do a quick sanity check.

In this section you will:

- Read the raw CSVs into a DataFrame with appropriate options (header, delimiter, types).
- Inspect basic schema and row counts.
- Look for obvious data quality issues (missing columns, strange encodings, unexpected file names).

This is a good moment to:

- Note any quirks you may need to handle in your own feature engineering or ML models.
- Decide which time ranges or subsets your team will focus on for the hackathon.


In [0]:
# Load
trips = pd.read_parquet(trip_data_dir / "mobi_trips.parquet")
stations = pd.read_parquet(trip_data_dir / "mobi_stations.parquet")
mobi_site = pd.read_parquet(site_data_dir / "mobibikes_ca_content.parquet")

logger.success(f"Trips loaded: {len(trips):,}")
logger.success(f"Stations loaded: {len(stations)}")
logger.success(f"Site pages available: {len(mobi_site)}")

## Create Bronze Tables

Bronze tables mirror the raw data as closely as possible while making it easier to query from SQL, Genie rooms, and notebooks.

In this section we:

- Write the raw trip DataFrame into a **bronze Delta table** in Unity Catalog.
- Preserve all raw columns and minimal transformations (e.g. consistent column naming, basic type casts).
- Optionally create bronze tables for any additional raw inputs (stations, site content, etc.).

At the end of this step you should have:

- A `bronze_trips` table in your chosen `catalog.schema`.
- A stable location you can use for exploration, feature building, and downstream silver pipelines.

> Treat bronze as your “single source of raw truth” — downstream tables can be rebuilt from it if needed.


In [0]:
trips_pq = str(trip_data_dir / "mobi_trips.parquet")
stations_pq = str(trip_data_dir / "mobi_stations.parquet")
site_pq = str(site_data_dir / "mobibikes_ca_content.parquet")

(
    spark.read.parquet(trips_pq)
    .write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", True)
    .saveAsTable(f'`{CATALOG}`.`{SCHEMA}`.`bronze_trips`')
)

(
    spark.read.parquet(stations_pq)
    .write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", True)
    .saveAsTable(f'`{CATALOG}`.`{SCHEMA}`.`bronze_stations`')
)

(
    spark.read.parquet(site_pq)
    .write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", True)
    .saveAsTable(f'`{CATALOG}`.`{SCHEMA}`.`bronze_site`')
)

## Create Silver Tables

### What Are Silver Tables?

Databricks workflows commonly organize data into three layers:

- **Bronze**  
  Raw or minimally processed data — faithful to the source, high-volume, low-governance.

- **Silver**  
  Cleaned, structured, well-typed, and analytics/ML-ready data.  
  Columns are standardized. Types are consistent. IDs are stable. Missing or inconsistent values are handled.

- **Gold** (not used in this repo)  
  Business-level aggregates, feature tables, or curated outputs.

In this project, you will create **three Silver tables**, each with a different purpose:

1. **`silver_trips`** — enriched, normalized trip history  
2. **`silver_stations`** — a cleaned station reference table  
3. **`silver_site_pages`** — structured website content ready for embeddings and vector search

Silver tables matter because:

- They remove irregularities from raw input sources  
- They provide clean schemas for use by Genie, ML models, BI tools, and vector search  
- They offer consistent join keys across datasets  
- They are the *primary* tables most teams will query during the hackathon  

Think of silver tables as your **default working layer** —  
Bronze is for ingestion, Silver is for intelligence.



In [0]:
# Inspect bronze tables before silver transforms

def summarize_columns(df: pd.DataFrame, label: str) -> pd.DataFrame:
    """Return a sorted DataFrame detailing column names, dtypes, and null counts."""
    summary = pd.DataFrame(
        {
            "column": df.columns,
            "dtype": df.dtypes.astype(str),
            "null_count": df.isna().sum(),
        }
    )
    logger.info(f"{label} has {len(df):,} rows")
    return summary.sort_values("column").reset_index(drop=True)

bronze_trips_summary = summarize_columns(trips, "bronze_trips")
bronze_stations_summary = summarize_columns(stations, "bronze_stations")
bronze_site_summary = summarize_columns(mobi_site, "bronze_site")

display(bronze_trips_summary)
display(bronze_stations_summary)
display(bronze_site_summary)


In [0]:
# Helper utilities for silver preparation

def clean_station_label_col(column: F.Column) -> F.Column:
    """Return a Spark column with 4-digit numeric prefixes removed and whitespace normalized."""
    return F.regexp_replace(
        F.regexp_replace(F.trim(column), r"^\s*\d{4}[\s\-:]*", ""), r"\s+", " "
    )


def normalize_station_key_col(column: F.Column) -> F.Column:
    """Create a lowercase, single-space station key suitable for joining."""
    return F.lower(F.regexp_replace(F.trim(column), r"\s+", " "))


def coerce_flag_col(column: F.Column) -> F.Column:
    """Convert yes/no style strings to a nullable boolean Spark column.

    TODO: Extend with additional localized values if they appear in new extracts.
    """
    normalized = F.lower(F.trim(column))
    truthy = ["yes", "true", "1", "y"]
    falsy = ["no", "false", "0", "n"]
    return (
        F.when(normalized.isin(*truthy), F.lit(True))
        .when(normalized.isin(*falsy), F.lit(False))
        .otherwise(F.lit(None))
    )


## Create Silver Stations Table

The raw GBFS feeds provide a mixture of static station metadata (“station information”) and dynamic availability snapshots (“station status”).  
To simplify downstream use, we consolidate these sources into a clean and consistent **`silver_stations`** reference table.

This table includes:

- Station IDs  
- Human-readable names  
- Latitude/longitude coordinates  
- Dock capacity  
- Normalized station names/labels  
- Any useful attributes extracted from scraped pages (if applicable)

Why this matters:

- Station names are inconsistent in the raw data (prefixes, formatting issues, multiple naming conventions)
- Trip datasets rely on station names/IDs for joins — consistency is critical
- Mapping, ML features, and agent workflows depend on precise geospatial information

`silver_stations` acts as the **source of truth** for anything involving station identity, location, or metadata.


In [0]:
# Prepare silver_stations DataFrame using Spark
bronze_stations_sdf = spark.table(f'`{CATALOG}`.`{SCHEMA}`.`bronze_stations`')

silver_stations = (
    bronze_stations_sdf
    .withColumn("station_name", clean_station_label_col(F.col("name")))
    .withColumn("station_key", normalize_station_key_col(F.col("name")))
    .withColumn("station_pk", F.col("station_id"))
    .dropDuplicates(["station_id"])
)
# TODO: capture additional station metadata (e.g., capacity) if required downstream.
logger.success(f"Prepared silver_stations with {silver_stations.count()} rows")
display(silver_stations.limit(5))


## Create Silver Trips Table

Our bronze trips table mirrors the raw CSVs, which is great for fidelity but not ideal for everyday analytics or ML. Silver tables give you a cleaner, more opinionated view.

In this section we:

- Transform `bronze_trips` into a **`silver_trips`** table designed for exploration, modelling, and Genie.
- Apply a focused set of transformations:
  - **Normalize columns** – consolidate duplicate or inconsistent names, cast metrics to numeric types, and standardize timestamps.
  - **Derive flags and features** – e.g. a single, trusted indicator for electric bikes.
  - **Standardize station names/IDs** – strip numeric prefixes where appropriate and ensure both start and end stations are easy to join.
  - **Assign primary keys** – create a deterministic `trip_id` using a window function so every trip row has a unique identifier.

You should:

- Inspect sample rows from both bronze and silver to see exactly what changed.
- Decide whether to add your own features (time-of-day, day-of-week, weather joins, etc.) either here or in your own notebooks.

> Think of `silver_trips` as the default table your team uses for queries, dashboards, agents, and ML — bronze is there when you need to go back to the raw source.

In [0]:
# Load bronze trips as the starting point for the silver transformation
bronze_trips_sdf = spark.table(f'`{CATALOG}`.`{SCHEMA}`.`bronze_trips`')
silver_trips = bronze_trips_sdf
logger.info(f"Loaded bronze_trips rows: {silver_trips.count()}")
display(silver_trips.limit(5))

In [0]:
# Step 1: Coalesce duplicate membership columns
if "memebership_type" in bronze_trips_sdf.columns:
    if "membership_type" in silver_trips.columns:
        silver_trips = silver_trips.withColumn(
            "membership_type",
            F.coalesce(F.col("membership_type"), F.col("memebership_type")),
        )
    else:
        silver_trips = silver_trips.withColumn("membership_type", F.col("memebership_type"))
    silver_trips = silver_trips.drop("memebership_type")

logger.info(f"After membership cleanup: {silver_trips.count()} rows")
display(silver_trips.select("membership_type").limit(5))


In [0]:
# Step 2: Normalize stopover duration values
if "stopover_duration" in silver_trips.columns:
    if "stopover_duration_sec" in silver_trips.columns:
        silver_trips = silver_trips.withColumn(
            "stopover_duration_sec",
            F.coalesce(F.col("stopover_duration_sec"), F.col("stopover_duration")),
        )
    else:
        silver_trips = silver_trips.withColumn(
            "stopover_duration_sec", F.col("stopover_duration").cast("double")
        )
    silver_trips = silver_trips.drop("stopover_duration")

logger.info(f"After stopover normalization: {silver_trips.count()} rows")
display(silver_trips.select("stopover_duration_sec").limit(5))


In [0]:
# Step 3: Apply numeric casts and derived distance
numeric_casts = {
    "bike_id": "bigint",
    "covered_distance_m": "double",
    "duration_sec": "double",
    "stopover_duration_sec": "double",
    "number_of_stopovers": "double",
    "departure_slot": "double",
    "return_slot": "double",
    "lock_duration_sec": "double",
    "number_of_bike_locks": "double",
    "number_of_bike_stopovers": "double",
    "departure_battery_voltage_mv": "double",
    "return_battery_voltage_mv": "double",
}
for column, dtype in numeric_casts.items():
    if column in silver_trips.columns:
        silver_trips = silver_trips.withColumn(column, F.col(column).cast(dtype))

if "covered_distance_m" in silver_trips.columns:
    silver_trips = silver_trips.withColumn(
        "covered_distance_km", F.col("covered_distance_m") / F.lit(1000.0)
    )

logger.info(f"After numeric coercion: {silver_trips.count()} rows")
display(silver_trips.select(*[c for c in ["covered_distance_m", "covered_distance_km", "duration_sec"] if c in silver_trips.columns]).limit(5))


In [0]:
# Step 4: Cast departure and return timestamps (source data already hourly)
# Handle slash and dash formats plus minute-only timestamps without seconds

from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
import pandas as pd
from pyspark.sql.functions import pandas_udf

# Pandas UDF fallback
@pandas_udf(TimestampType())
def infer_ts(col: pd.Series) -> pd.Series:
    """Fallback parser using pandas.to_datetime() with inference."""
    return pd.to_datetime(col, errors="coerce", infer_datetime_format=True)

# Fast native parse
def fast_timestamp(col):
    """Lightweight parser for most rows."""
    c = F.trim(F.col(col))
    c = F.regexp_replace(c, r"[./]", "-")  # normalize separators
    # reorder M-D-Y -> Y-M-D
    c = F.regexp_replace(c, r"^([0-9]{1,2})-([0-9]{1,2})-([0-9]{4})", r"\3-\1-\2")
    return F.try_to_timestamp(c)

for raw, target in [("departure", "departure_time"), ("return", "return_time")]:
    if raw in silver_trips.columns:
        # fast native parse
        silver_trips = silver_trips.withColumn(target, fast_timestamp(raw))
        # pandas fallback only for nulls
        silver_trips = silver_trips.withColumn(
            target,
            F.when(F.col(target).isNull(), infer_ts(F.col(raw))).otherwise(F.col(target))
        )
        silver_trips = silver_trips.drop(raw)

for colname in ["departure_time", "return_time"]:
    if colname in silver_trips.columns:
        silver_trips = silver_trips.withColumn(colname, fast_timestamp(colname))
        silver_trips = silver_trips.withColumn(
            colname,
            F.when(F.col(colname).isNull(), infer_ts(F.col(colname))).otherwise(F.col(colname))
        )

# Add date based partitioning columns for later
if "departure_time" in silver_trips.columns:
    silver_trips = (
        silver_trips
        .withColumn("departure_year",  F.year("departure_time"))
        .withColumn("departure_month", F.month("departure_time"))
    )

display(silver_trips.select("departure_time", "return_time", "departure_year", "departure_month").limit(5))

In [0]:
# Step 5: Consolidate electric bike indicators (coalesce known boolean columns)
flag_columns = [c for c in ["electric_bike", "electric"] if c in silver_trips.columns]

if flag_columns:
    silver_trips = silver_trips.withColumn(
        "is_electric_bike",
        F.coalesce(*[F.col(c).cast("boolean") for c in flag_columns]),
    )
else:
    silver_trips = silver_trips.withColumn("is_electric_bike", F.lit(None).cast("boolean"))

for column in flag_columns:
    silver_trips = silver_trips.drop(column)

logger.info(f"After electric flag normalization: {silver_trips.count()} rows")
display(silver_trips.select("is_electric_bike").limit(5))


In [0]:
# Step 6: Clean station names (strip numeric prefixes, collapse whitespace)
for prefix in ["departure", "return"]:
    raw_column = f"{prefix}_station"
    if raw_column in silver_trips.columns:
        cleaned = clean_station_label_col(F.col(raw_column))
        silver_trips = silver_trips.withColumn(f"{prefix}_station_name", cleaned)
        silver_trips = silver_trips.drop(raw_column)

logger.info(f"After station name cleanup: {silver_trips.count()} rows")
display(
    silver_trips.select(
        *[
            c
            for c in [
                "departure_station_name",
                "return_station_name",
            ]
            if c in silver_trips.columns
        ]
    ).limit(5)
)


In [0]:
# Step 7: Attach station IDs using normalized names
dep_lookup = silver_stations.select(
    F.col("station_key").alias("dep_station_key"),
    F.col("station_id").alias("departure_station_id"),
)
ret_lookup = silver_stations.select(
    F.col("station_key").alias("ret_station_key"),
    F.col("station_id").alias("return_station_id"),
)

silver_trips = silver_trips.join(
    dep_lookup,
    normalize_station_key_col(F.col("departure_station_name")) == dep_lookup["dep_station_key"],
    "left",
).drop("dep_station_key")

silver_trips = silver_trips.join(
    ret_lookup,
    normalize_station_key_col(F.col("return_station_name")) == ret_lookup["ret_station_key"],
    "left",
).drop("ret_station_key")

logger.info(f"After station ID join: {silver_trips.count()} rows")
display(
    silver_trips.select(
        *[
            c
            for c in [
                "departure_station_name",
                "departure_station_id",
                "return_station_name",
                "return_station_id",
            ]
            if c in silver_trips.columns
        ]
    ).limit(5)
)


In [0]:
# Step 8: Finalize trip_id and column ordering (simple fast version)

# Generate a unique ID per row
silver_trips = silver_trips.withColumn("trip_id", F.monotonically_increasing_id())

# Preserve original column ordering logic
trip_columns_order = [
    "trip_id",
    "departure_time",
    "return_time",
    "bike",
    "is_electric_bike",
    "membership_type",
    "covered_distance_m",
    "covered_distance_km",
    "duration_sec",
    "stopover_duration_sec",
    "number_of_stopovers",
    "number_of_bike_stopovers",
    "departure_slot",
    "return_slot",
    "lock_duration_sec",
    "number_of_bike_locks",
    "departure_temperature_c",
    "return_temperature_c",
    "departure_battery_voltage_mv",
    "return_battery_voltage_mv",
    "departure_station_name",
    "departure_station_id",
    "return_station_name",
    "return_station_id",
    "source_file",
    "formula",
    "manager",
]

existing_trip_columns = [c for c in trip_columns_order if c in silver_trips.columns]
remaining_columns = [c for c in silver_trips.columns if c not in existing_trip_columns]
silver_trips = silver_trips.select(*(existing_trip_columns + remaining_columns))

logger.success(f"Prepared silver_trips with {silver_trips.count()} rows")
display(silver_trips.limit(5))


## Create Silver Site Table

The scraped website pages contain rich textual content that can power:

- Semantic search  
- Retrieval-augmented generation  
- Question answering in Genie rooms  
- Summaries, descriptions, and contextual reasoning for ML or agent workflows  

The raw scraped output can be noisy, so here we build **`silver_site_pages`**, which includes:

- Clean page titles  
- Normalized URLs  
- Markdown-formatted content (`content_md`)  
- Page categories or implicit tags (optional)

This table is structured specifically to support **Vector Search**, where each row becomes:

- An embedding  
- A retrievable document  
- A source for agent answers or table function inputs  

If your team plans to use GenAI or build QA-style features, this table will be one of your most important assets.


In [0]:
# Prepare silver_site DataFrame and assign primary key
bronze_site_sdf = spark.table(f'`{CATALOG}`.`{SCHEMA}`.`bronze_site`')

site_window = Window.orderBy("url")
silver_site = (
    bronze_site_sdf
    .withColumn("scraped_at", F.to_timestamp("scraped_at"))
    .dropDuplicates(["url"])
    .withColumn("site_page_id", F.row_number().over(site_window).cast("bigint"))
)
# TODO: revisit primary key strategy if additional natural keys become available.

logger.success(f"Prepared silver_site with {silver_site.count()} pages")
display(silver_site.limit(5))


In [0]:
# Persist silver tables to Unity Catalog Delta

logger.info("Repartitioning silver_trips by year and month")
silver_trips = silver_trips.repartition("departure_year", "departure_month")

# Drop existing tables (if any) to avoid slow schema reconciliation
for t in ["silver_trips", "silver_stations", "silver_site"]:
    spark.sql(f"DROP TABLE IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{t}`")

logger.info(f"Writing silver_trips to `{CATALOG}`.`{SCHEMA}`.silver_trips")
(
    silver_trips.write
    .format("delta")
    .partitionBy("departure_year", "departure_month")
    .saveAsTable(f"`{CATALOG}`.`{SCHEMA}`.`silver_trips`")
)

logger.info(f"Writing silver_stations to `{CATALOG}`.`{SCHEMA}`.silver_stations")
(
    silver_stations.write
    .format("delta")
    .saveAsTable(f"`{CATALOG}`.`{SCHEMA}`.`silver_stations`")
)

logger.info(f"Writing silver_site to `{CATALOG}`.`{SCHEMA}`.silver_site")
(
    silver_site.write
    .format("delta")
    .saveAsTable(f"`{CATALOG}`.`{SCHEMA}`.`silver_site`")
)

logger.success("Silver tables materialized")

silver_trips.printSchema()
silver_stations.printSchema()
silver_site.printSchema()